# Get summary GA

In [1]:
import pandas as pd
from pathlib import Path
from collections import defaultdict
from tabulate import tabulate
from rdkit import Chem
import shutil

from support_mvp.backup_plot_diagram.plot_diagram_schrock import (
    HARTREE2KCAL,
    base,
    collection,
    dicts,
    get_diagram_dicts,
    get_diagram_dicts_return_gibbs
)

oneup


In [2]:

smiles = ['CCCN1CCCC(N)(C2CCCC2)C1','CCCOC(=O)NCC1(N)CCCCC1', 'NC1(CCCCCl)CCCCC1', 'CC(C)(N)CCC1CCCCC1', 'CC(N)Cc1ccc(Cc2ccccc2)cc1', 'CC(N)Cc1ccc(CCl)cc1', 'CC(=O)Nc1ccc(F)c(C(=O)Cc2ccnc(N)c2)c1', 'C=C(C)CC(C)(C)N', 'CC(=O)c1ccc(F)c(C(=O)CN)c1', 'CC(=O)c1ccc(F)c(C(=O)N=CN)c1', 'CC(=O)c1cc(O)c(C(=O)CN)cc1O', 'N#Cc1ccnc(C#N)c1N', 'N#Cc1ccnc(C(=O)Cl)c1N', 'N#Cc1cnc(C#N)c(N)c1', 'N#Cc1ncc(CC(=O)O)cc1N', 'N#Cc1ncc(C(=O)CO)cc1N', 'Nc1ccccc1N=CC(=O)Cl', 'NCCc1ncnc2ccccc12', 'NCC=Cc1ncnc2ccccc12', 'NC=NC(=O)c1cccc(Br)c1', 'NCCCc1ncnc2ccccc12', 'NCCCOc1ccnc2cccnc12', 'NCCCOc1ncnc2cccnc12', 'NCCOC(=O)c1ncnc2ccccc12']

pkls = [
"/home/magstr/Documents/GB_GA/notebooks/df_nh3ton2.pkl",
"/home/magstr/Documents/GB_GA/notebooks/df_n2binding.pkl",
"/home/magstr/Documents/GB_GA/notebooks/df_reduction_step.pkl",
]

base = Path("/home/magstr/Documents/")
path_dir = base / "schrock/diagrams_schrock/dft/b3lyp_candidates"

In [3]:
# Canidates i aready have singlepoints for:
already_sp = ['C=C(C)CC(C)(C)N', 'N#Cc1ncc(C(=O)CO)cc1N', 'NCCc1ncnc2ccccc12', 'NCCCOc1ccnc2cccnc12', 'CC(N)Cc1ccc(Cc2ccccc2)cc1', 'CC(C)(N)CCC1CCCCC1', 'CC(N)Cc1ccc(CCl)cc1', 'N#Cc1cnc(C#N)c(N)c1', 'N#Cc1ccnc(C#N)c1N', 'N#Cc1ccnc(C(=O)Cl)c1N', 'CCCOC(=O)NCC1(N)CCCCC1','NCCOC(=O)c1ncnc2ccccc12']

In [4]:
for_dft = list(set(smiles) - set(already_sp))
print(for_dft)

['NCCCc1ncnc2ccccc12', 'CC(=O)c1ccc(F)c(C(=O)CN)c1', 'CCCN1CCCC(N)(C2CCCC2)C1', 'NC=NC(=O)c1cccc(Br)c1', 'NCC=Cc1ncnc2ccccc12', 'NCCCOc1ncnc2cccnc12', 'NC1(CCCCCl)CCCCC1', 'Nc1ccccc1N=CC(=O)Cl', 'CC(=O)c1cc(O)c(C(=O)CN)cc1O', 'N#Cc1ncc(CC(=O)O)cc1N', 'CC(=O)c1ccc(F)c(C(=O)N=CN)c1', 'CC(=O)Nc1ccc(F)c(C(=O)Cc2ccnc(N)c2)c1']


## Create singlepoint folders for the remaining 12 for_dft

## Create summary dict

In [13]:
d3 = {}
summary_dict = {}
for elem in already_sp:
    
    print(elem)
    m = Chem.MolFromSmiles(elem)
    # Replace tricky primary amines in the frag:
    prim_match = Chem.MolFromSmarts("[NX3;H2]")
    rm = Chem.ReplaceSubstructs(
                                m,
                                prim_match,
                                Chem.MolFromSmiles("[1*]"),
                                replaceAll=True,
                            )[0]
    new_smiles=Chem.MolToSmiles(rm)
    
    for f in pkls:
        mol_df = pd.read_pickle(f)
    # Get the scoring
        tmp = mol_df.loc[mol_df["smiles"] == elem]
        if not tmp.empty:
            summary_dict[new_smiles] = {'scoring function':tmp["scoring_function"].values[0]}

            break
        else:
            val = ""
            continue

    folders = sorted(path_dir.rglob(f"*{elem}"))
    
    options = {
                "exchange": True,
                "thermal_correction": True,
                'diagrams': [
                    [
                        folders[0],
                        "orca.out",
                        "FINAL SINGLE",
                        dicts["parts_orca_b3lyp_sp"],
                        Path(f'/home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/xtb_vibrations/{elem}')
                    ],
                ]
            }
    options = defaultdict(lambda: False, options)
    compare_data, delta_gibbs = get_diagram_dicts_return_gibbs(options)
    
    options_ref = {
                "exchange": True,
                "thermal_correction": True,
                'diagrams': [
                    [
                        base / "schrock/diagrams_schrock/dft/b3lyp_sp",
                        "orca.out",
                        "FINAL SINGLE",
                        dicts["parts_orca_b3lyp_sp"],
                        Path("/home/magstr/Documents/schrock/diagrams_schrock/vibrations_original_xtb/")
                    ],
                ]
            }
    

    options_ref = defaultdict(lambda: False, options_ref)
    compare_data_ref, delta_gibbs_ref = get_diagram_dicts_return_gibbs(options_ref)
    
    
    summary_dict[new_smiles]['NH3toN2'] = compare_data[elem]['Mo_N2_exchange']-compare_data[elem]['Mo_NH3']
    summary_dict[new_smiles]['NH3toN2_NH3'] = compare_data[elem]['Mo_N2_NH3']-compare_data[elem]['Mo_NH3']
    summary_dict[new_smiles]['NH3+toNH3'] = compare_data[elem]['Mo_NH3']-compare_data[elem]['Mo_NH3+']
    
    
    delta_gibbs_ref.pop('Mo_N2_exchange')
    delta_gibbs_ref.pop('Mo_N2_bare')
    delta_gibbs_ref.pop('Mo')
    
    delta_gibbs.pop('Mo_N2_exchange')
    delta_gibbs.pop('Mo_N2_bare')
    delta_gibbs.pop('Mo')
    
    
    # Find step that increased most compared to refference
    d3 = {key: delta_gibbs[key] - delta_gibbs_ref.get(key, 0) for key in delta_gibbs}
    summary_dict[new_smiles]['max_change'] = d3[max(d3, key=d3.get)]

C=C(C)CC(C)(C)N
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/C=C(C)CC(C)(C)N
NH3->N2 : -9.27844085264951
NH3->N2_NH3 : 3.490633698645979
NH3+->NH3 : 9.894694034010172
lol
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_sp
NH3->N2 : 8.48247880116105
NH3->N2_NH3 : -2.072646460495889
NH3+->NH3 : 1.5236350516788661
lol
N#Cc1ncc(C(=O)CO)cc1N
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/N#Cc1ncc(C(=O)CO)cc1N
NH3->N2 : -7.442760905250907
NH3->N2_NH3 : -17.65318280365318
NH3+->NH3 : -2.2510269307531416
lol
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_sp
NH3->N2 : 8.48247880116105
NH3->N2_NH3 : -2.072646460495889
NH3+->NH3 : 1.5236350516788661
lol
NCCc1ncnc2ccccc12
Doing diagram /home/magstr/Documents/schrock/diagrams_schrock/dft/b3lyp_candidates/NCCc1ncnc2ccccc12
NH3->N2 : -1.972812126390636
NH3->N2_NH3 : 4.720123135019094
NH3+->NH3 : 11.319995245430619
lol
Doing diagr

In [14]:
for k, v in summary_dict.items():
        print(k,v)

[1*]C(C)(C)CC(=C)C {'scoring function': 'NH3 -> N2', 'NH3toN2': -21.012, 'NH3toN2_NH3': 15.447000000000031, 'NH3+toNH3': 8.890000000000015, 'max_change': 12.758}
[1*]c1cc(C(=O)CO)cnc1C#N {'scoring function': 'NH3+ -> NH3', 'NH3toN2': -20.427999999999997, 'NH3toN2_NH3': -7.225999999999999, 'NH3+toNH3': -0.9590000000000032, 'max_change': 24.686}
[1*]CCc1ncnc2ccccc12 {'scoring function': 'NH3 -> NH3 - N2', 'NH3toN2': -14.160000000000025, 'NH3toN2_NH3': 15.760999999999996, 'NH3+toNH3': 13.699999999999989, 'max_change': 20.641000000000002}
[1*]CCCOc1ccnc2cccnc12 {'scoring function': 'NH3 -> NH3 - N2', 'NH3toN2': -17.998999999999995, 'NH3toN2_NH3': 6.870999999999981, 'NH3+toNH3': 15.249000000000024, 'max_change': 23.226000000000006}
[1*]C(C)Cc1ccc(Cc2ccccc2)cc1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -22.30600000000001, 'NH3toN2_NH3': 16.97199999999998, 'NH3+toNH3': 11.63300000000001, 'max_change': 13.118000000000002}
[1*]C(C)(C)CCC1CCCCC1 {'scoring function': 'NH3 -> N2', 'NH3toN2': -2

In [15]:
def create_latex_tables():

    # Example code:
    # https://colab.research.google.com/drive/1Iq10lHznMngg1-Uoo-QtpTPii1JDYSQA?usp=sharing#scrollTo=K7NNR1Vg40Vo

    # Simple header
    headers = ["Smiles","NH3toN2", 'NH3+toNH3', 'NH3toN2_NH3', 'Max \delta\delta E']
    
    rows = [ ]
    for k, v in summary_dict.items():
        rows.append([k,v['NH3toN2'],v['NH3+toNH3'],v['NH3toN2_NH3'], v['max_change']])


    print("Tabulate Table:")
    print(tabulate(rows, headers=headers))

    # Formatting option: floatfmt=".3f"
    print("\nTabulate Latex:")
    print(tabulate(rows, headers=headers, tablefmt="latex", floatfmt=".2f"))

    return

In [16]:
create_latex_tables()

Tabulate Table:
Smiles                          NH3toN2    NH3+toNH3    NH3toN2_NH3    Max \delta\delta E
----------------------------  ---------  -----------  -------------  --------------------
[1*]C(C)(C)CC(=C)C              -21.012        8.89          15.447                12.758
[1*]c1cc(C(=O)CO)cnc1C#N        -20.428       -0.959         -7.226                24.686
[1*]CCc1ncnc2ccccc12            -14.16        13.7           15.761                20.641
[1*]CCCOc1ccnc2cccnc12          -17.999       15.249          6.871                23.226
[1*]C(C)Cc1ccc(Cc2ccccc2)cc1    -22.306       11.633         16.972                13.118
[1*]C(C)(C)CCC1CCCCC1           -23.655       10.327         13.649                 9.388
[1*]C(C)Cc1ccc(CCl)cc1          -22.133       11.575         14.516                15.187
[1*]c1cc(C#N)cnc1C#N             14.595      -17.579         15.359                35.794
[1*]c1c(C#N)ccnc1C#N            -32.464        7.405         -8.378                2